<a href="https://colab.research.google.com/github/seongyeon01/Statistical-Data-Artificial-Intelligence-Contest/blob/main/%ED%86%B5%EA%B3%84%EC%B2%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install konlpy

from tqdm import tqdm
import time
import numpy as np
import pandas as pd
import sentencepiece as spm
from typing import Text
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from konlpy.tag import Okt
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, LSTM, Embedding, Bidirectional
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from collections import Counter
from nltk.tokenize import word_tokenize
from keras.models import Sequential
from keras.layers import Flatten, Dropout
from keras.models import load_model
from keras.callbacks import EarlyStopping
import nltk as nltk
nltk.download('punkt')
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train = pd.read_csv("/content/drive/Shareddrives/공모전/[통계청] 2022 통계데이터 인공지능 활용대회/data/train.txt", encoding='cp949',  sep='|')
test = pd.read_csv('/content/drive/Shareddrives/공모전/[통계청] 2022 통계데이터 인공지능 활용대회/data/test.csv', encoding='cp949')

#전처리1 불용어제거
okt = Okt()

def pre_concat(train):
        # text columns 병합 함수
    train= train.fillna('')
    train['unk']=' '
    train['text']= train['text_obj']+train['unk']+train['text_mthd']+train['unk']+train['text_deal']
    train = train.drop(columns=['text_obj','unk','text_mthd','text_deal'])
    print(train.head())
    return train

def pre_del_sth(x):
        # 부호, 오타, 불용어 제거 함수
    typing_error = ['ㄱ','ㄴ','ㄷ','ㄹ','ㅁ','ㅂ','ㅅ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ','ㅋ','ㅌ','ㅊ','ㅍ'
                    'ㅃ','ㅉ','ㄸ','ㄲ','ㅆ','ㅓ','ㅏ','ㅣ','ㅜ','ㅡ','ㅗ','ㅛ','ㅕ','ㅑ','ㅐ','ㅒ','ㅔ','ㅖ','ㅢ','ㅙ','ㅞ','ㅝ','ㅟ']
    signs = ['!','@','#','$','%','^','&','*','(',')','_','-','+','=','[',']','{','}',':',';','\"','\'','\','<','>',','.','?','?','~','|']
    for c in typing_error :
      x['text'] = x['text'].apply(lambda x: x.replace(c, ''))
    for c in signs :
      x['text'] = x['text'].apply(lambda x: x.replace(str(c), '').replace(',',' '))
    x['text'] = x['text'].apply(lambda x: x.replace('에서', ' ').replace('으로',' ').replace('에게',' ').replace('하여',' ').replace('및',' ').replace('해서', ' '))
    x['text']= x['text'].apply(lambda x: x if '원하는' not in x else x.replace('하는', ' '))
    return x

def pre_del_josa(x):
        # 특정 조사 제거 함수
    x_dict = x['text'].to_dict()
    for i in tqdm(range(0, len(x_dict))):
      _morphs = okt.pos(x_dict[i], norm=False, stem=True)
      words = []
      for word, pos in _morphs:
        if pos == 'Josa' :
          if word == '에도' or word == '요' or word == '도' or word == '은' or word == '가' or word == '이':
            pass
          else:
            x_dict[i]= x_dict[i].replace(word, ' ').replace('에',' ')
        else:
          pass
    x['text']= pd.DataFrame([x_dict]).T
    return x

In [ ]:
train = pre_concat(train)
train = pre_del_sth(train)
train = pre_del_josa(train)

test = pre_concat(test)
test = pre_del_sth(test)
test = pre_del_josa(test)

print(train.head())
print(test.head())

        AI_id digit_1  digit_2  digit_3                         text
0  id_0000001       S       95      952        카센터에서 자동차부분정비 타이어오일교환
1  id_0000002       G       47      472      상점내에서 일반인을 대상으로 채소.과일판매
2  id_0000003       G       46      467  절단하여사업체에도매 공업용고무를가지고 합성고무도매
3  id_0000004       G       47      475         영업점에서 일반소비자에게 열쇠잠금장치
4  id_0000005       Q       87      872     어린이집 보호자의 위탁을 받아 취학전아동보육


100%|██████████| 1000000/1000000 [29:46<00:00, 559.62it/s]


       AI_id digit_1 digit_2 digit_3                        text
0  id_000001                               치킨전문점에서 고객의주문에의해 치킨판매
1  id_000002                               산업공구 다른 소매업자에게 철물 수공구
2  id_000003                                 절에서 신도을 대상으로 불교단체운영
3  id_000004                                   영업장에서 고객요구로 자동차튜닝
4  id_000005                          실내포장마차에서 접객시설을 갖추고 소주,맥주제공


100%|██████████| 100000/100000 [02:55<00:00, 570.13it/s]


        AI_id digit_1  digit_2  digit_3                        text
0  id_0000001       S       95      952        카센터  자동차부분정비 타이어오일교환
1  id_0000002       G       47      472        상점내  일반인  대상  채소과일판매
2  id_0000003       G       46      467  절단 사업체 도매 공업용 무 가지  합성 무도매
3  id_0000004       G       47      475          영업점  일반소비자  열쇠잠금장치
4  id_0000005       Q       87      872    어린이집 보호자  위탁  받아 취학전아동보육
       AI_id digit_1 digit_2 digit_3                       text
0  id_000001                               치킨전문점  고객 주문  해 치킨판매
1  id_000002                               산업공구 다른 소매업자  철물 수공구
2  id_000003                                  절  신도을 대상  불교단체운영
3  id_000004                                   영업장  고객요구  자동차튜닝
4  id_000005                          실내포장마차  접객시설  갖추고 소주 맥주제공


In [ ]:
#전처리2 subword Spacing(BPE모델 만들기)
train['text'] = train['text'].apply(lambda x: x.replace(' ', '/'))
test['text'] = test['text'].apply(lambda x: x.replace(' ', '/'))
train['text'].to_csv('/content/drive/MyDrive/2022/공모전/train_prepro.txt')

# 설정: Sentencepiece input parameter
templates= '--input={} \
--pad_id={} \
--bos_id={} \
--eos_id={} \
--unk_id={} \
--model_prefix={} \
--vocab_size={} \
--character_coverage={} \
--model_type={}'

#파일경로에 빈칸이 있으면 오류발생
train_input_file = '/content/drive/MyDrive/2022/공모전/train_prepro.txt'
pad_id= 0
vocab_size = 25000
prefix = 'tokenizer_1'
bos_id=1
eos_id=2
unk_id=3
character_coverage = 0.9995
model_type='unigram'
cmd= templates.format(train_input_file, pad_id, bos_id, eos_id, unk_id,prefix, vocab_size, 
                    character_coverage, model_type)
#BPE 분절하기
spm.SentencePieceTrainer.Train(cmd)
sp = spm.SentencePieceProcessor()
sp.Load('tokenizer_1.model')

True

In [ ]:
#전처리3 Spacing 함수 (sentencepiece tokenizing)
def spacing(x):
    x_dict = x['text'].to_dict()
    for i in tqdm(range(0,len(x_dict))):
      x_dict[i] = sp.EncodeAsPieces(x_dict[i])
      x_dict[i] = ' '.join(x_dict[i])
    x['text']= pd.DataFrame([x_dict]).T
    return x

train = spacing(train)
test = spacing(test)

print(train.head())
print(test.head())

In [ ]:
# Todo: Tokenizing - digit_1
'''전처리 후 토큰화 작업, 토큰화 함수 생성 (nltk의 tokenize 활용)'''

train['digit_1']= train['digit_1'].apply(lambda x: (ord(x)-64))

data= train['text']
target = train['digit_1']

# train_test_split
x_train, x_val, y_train, y_val = train_test_split(data, target, test_size = 0.1, random_state=12)
test_train = test['text']

# Tokenizing
def tokenizing(x):
      # Tokenizing 함수
    print(x)
    x_dict = x.to_dict()
    for c in tqdm(x_dict):
        x_dict[c] = word_tokenize(x_dict[c])
        temp = []
        for w in x_dict[c]:
            try:
                temp.append(word_to_index[w])
            except KeyError:
                pass
        x_dict[c] = temp
    x = pd.DataFrame([x_dict]).T
    print(x)
    return x

vocab = Counter()

x_dict = x_train.to_dict()
vocab_sorted = {}
for c in tqdm(x_dict):
    x_dict[c] = word_tokenize(x_dict[c])
    for word in x_dict[c]:
        vocab[word] = vocab[word] +1
vocab_sorted = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
print(vocab_sorted)

word_to_index = {}
i = 0
for (word, frequency) in vocab_sorted:
    if frequency > 5 and frequency< 700000:
        i += 1
        word_to_index[word] = i
print(word_to_index)

x_train = tokenizing(x_train)
x_val = tokenizing(x_val)
test_train = tokenizing(test_train)

100%|██████████| 900000/900000 [01:47<00:00, 8371.41it/s]


[('/', 4679282), ('▁', 899249), ('대상', 158155), ('갖추고', 125333), ('사업장', 116146), ('접객시설', 104304), ('고객', 102918), ('일반인', 87354), ('매장', 72847), ('음식점', 61315), ('일반소비자', 59540), ('요청', 58283), ('판매', 57369), ('소매', 52213), ('화물운송서비스', 40846), ('도매', 36187), ('개인택시', 34135), ('승객운송서비스', 34118), ('서비스', 29333), ('제공', 27213), ('영업장', 26295), ('일반인대상', 26286), ('용달화물자동차', 23502), ('사무실', 23322), ('등', 22420), ('제조', 20494), ('학원', 19244), ('받아', 18440), ('따라', 16542), ('계약', 16088), ('가게', 15408), ('개별화물자동차', 15337), ('소매업자', 14917), ('부동산', 14226), ('미용실', 14001), ('일반고객', 13771), ('커피', 13404), ('해', 12861), ('교회', 11873), ('소비자', 11853), ('일반', 11645), ('시설', 11588), ('주점', 11300), ('가공', 10272), ('산업사용자', 10221), ('종교서비스', 9939), ('파마', 9632), ('염색', 9531), ('학생', 9447), ('맥주', 9392), ('교육', 9356), ('중개', 9087), ('뢰', 9075), ('고객대상', 9063), ('편의점', 9024), ('절단', 8962), ('현장', 8843), ('자동차', 8722), ('가지', 8676), ('요구', 8609), ('어린이집', 7853), ('철', 7804), ('수리', 7691), ('조립', 7524), 

100%|██████████| 900000/900000 [01:54<00:00, 7852.52it/s]


                                    0
34488     [19, 163, 1, 114, 1393, 12]
395706  [1222, 374, 398, 418, 1, 238]
120781          [2269, 24, 11, 11077]
684091             [19, 5, 1, 94, 18]
157096             [107, 75, 36, 906]
...                               ...
564465                     [2132, 14]
412925                 [551, 878, 11]
255366              [25, 47, 1, 2506]
759451             [22, 77, 3755, 77]
472907   [53, 9, 278, 520, 1100, 904]

[900000 rows x 1 columns]
853800                     ▁ 사업장 / / 일반인대상 / / 김치전골 / 청국장판매
542136                    ▁ 조세 / 세무관련업무 / 의뢰 / / 받아서 / 기장대리
239719                                ▁ 사업장 / / 시설갖추고 / 해산물
557263                ▁ 보험회사 / / 일반인 / / 대상 / / 자동차보험 판매 중개
891459                         ▁ 화원 / 일반인 / / 대상 / / 꽃 / 화분
                                ...                        
112252                             ▁ 주점 / / 접객요원 두 / / 맥주양주
236781    ▁ 폴리 / 틸렌 PE 를가지 / / 절단성형 / / 해상 부 유 구조물 플라스틱 ...
212026                              ▁

100%|██████████| 100000/100000 [00:12<00:00, 7993.56it/s]


                                                        0
853800                              [3, 20, 13854, 13541]
542136                         [1693, 1276, 71, 343, 573]
239719                                    [3, 1474, 1193]
557263                          [918, 6, 1, 5486, 11, 50]
891459                              [698, 6, 1, 246, 853]
...                                                   ...
112252                                [41, 72, 949, 1613]
236781  [944, 8389, 1332, 7515, 11189, 2989, 572, 618,...
212026                                 [180, 11, 19, 309]
200198     [2102, 667, 1400, 126, 51, 26, 121, 9592, 126]
425136                    [104, 925, 2096, 983, 1999, 11]

[100000 rows x 1 columns]
0             ▁ 치킨전문점 / / 고객 / 주문 / / 해 / 치킨판매
1              ▁ 산업공구 / 다른 / 소매업자 / / 철물 / 수공구
2                 ▁ 절 / / 신도을 / 대상 / / 불교단체 운영
3                     ▁ 영업장 / / 고객요구 / / 자동차튜닝
4        ▁ 실내포장마차 / / 접객시설 / / 갖추고 / 소주 / 맥주제공
                         ...              

100%|██████████| 100000/100000 [00:12<00:00, 7966.27it/s]


                                 0
0           [353, 5, 118, 36, 352]
1      [17082, 388, 31, 260, 1577]
2        [316, 21061, 1, 6172, 66]
3                 [19, 142, 12319]
4           [6320, 4, 2, 74, 1600]
...                            ...
99995          [3, 20, 1655, 1658]
99996               [158, 82, 333]
99997         [39, 317, 38, 13481]
99998            [3, 6, 137, 6767]
99999      [3, 128, 3228, 127, 80]

[100000 rows x 1 columns]


In [ ]:
print(x_train.head())
print(x_val.head())
print(test_train.head())

In [ ]:
# Todo: Learning - digit_1
'''양방향 LSTM으로 학습 모델을 생성, 학습용 모델 함수 생성'''

max_features = 24000
text_max_words = 20

# 1. 전처리
x_train = sequence.pad_sequences(x_train[0], maxlen=text_max_words)
x_val = sequence.pad_sequences(x_val[0], maxlen=text_max_words)
y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)
test_train = sequence.pad_sequences(test_train[0], maxlen=text_max_words)

print(x_train)
print(y_train)

In [ ]:
def bilstm_model(d,num, x_train, y_train, x_val, y_val): 
        # 2. 모델 구성하기 함수
    es = EarlyStopping(monitor='val_loss', mode='min', patience=1, verbose=1)
    model = Sequential()
    model.add(Embedding(24000, 128, input_length=text_max_words))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.5))
    model.add(Dense(d, activation='sigmoid'))
        # 3. 모델 학습과정 설정하기
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        # 4. 모델 학습시키기
    hist = model.fit(x_train, y_train, epochs=7, callbacks=[es], batch_size=64, validation_data=[x_val, y_val])
    pred = model.predict(test_train)
    pred = np.argmax(pred, axis=1)
    return pred
        # 5. 모델 저장하기  
    list = ['model_1','model_2','model_3']
    with open(list[num-1], 'wb') as fw:
        pickle.dump(model, fw)

pred = bilstm_model(20, 1, x_train, y_train, x_val, y_val)
pred_1 = pred.tolist()
pred_1 = pd.DataFrame(data = pred_1, columns=['digit_1'])
print(pred_1.head())

pred_1.to_csv('/content/drive/MyDrive/2022/공모전/digit1_1.csv')

Epoch 1/7
14063/14063 [==============================] - 278s 19ms/step - loss: 0.0211 - accuracy: 0.9381 - val_loss: 0.0087 - val_accuracy: 0.9728
Epoch 2/7
14063/14063 [==============================] - 266s 19ms/step - loss: 0.0085 - accuracy: 0.9762 - val_loss: 0.0082 - val_accuracy: 0.9746
Epoch 3/7
14063/14063 [==============================] - 257s 18ms/step - loss: 0.0071 - accuracy: 0.9796 - val_loss: 0.0079 - val_accuracy: 0.9763
Epoch 4/7
14063/14063 [==============================] - 253s 18ms/step - loss: 0.0063 - accuracy: 0.9819 - val_loss: 0.0080 - val_accuracy: 0.9759
Epoch 4: early stopping
   digit_1
0        9
1        7
2       19
3       19
4        9


In [ ]:
# Todo: Tokenizing & Learning - digit_2
'''digit_1을 예측한 뒤 digit_2 예측에 활용'''

train_2 = train.copy()
test_2 = test.copy()
test_2['digit_1'] = pred_1['digit_1']
print(train_2.head())
print(test_2.head())

        AI_id  digit_1  digit_2  digit_3  \
0  id_0000001       19       95      952   
1  id_0000002        7       47      472   
2  id_0000003        7       46      467   
3  id_0000004        7       47      475   
4  id_0000005       17       87      872   

                                           text  
0                  ▁ 카센터 / / 자동차부분정비 / 타이어 오일교환  
1             ▁ 상점 내 / / 일반인 / / 대상 / / 채소과일 판매  
2  ▁ 절단 / 사업체 / 도매 / 공업용 / 무 / 가지 / / 합성 / 무 도매  
3                   ▁ 영업점 / / 일반소비자 / / 열쇠 잠금장치  
4          ▁ 어린이집 / 보호자 / / 위탁 / / 받아 / 취학전아동보육  
       AI_id  digit_1 digit_2 digit_3                                   text
0  id_000001        9                       ▁ 치킨전문점 / / 고객 / 주문 / / 해 / 치킨판매
1  id_000002        7                        ▁ 산업공구 / 다른 / 소매업자 / / 철물 / 수공구
2  id_000003       19                           ▁ 절 / / 신도을 / 대상 / / 불교단체 운영
3  id_000004       19                               ▁ 영업장 / / 고객요구 / / 자동차튜닝
4  id_000005        9                  ▁ 실내포장마차 / 

In [ ]:
def plus_catg(x,num):
        # digit_?을 text에 append하는 함수
    x_dict = x.to_dict()
    for i in range(0,len(x_dict['text'])):
        temp1 = [chr(x_dict[num][i]+64)]
        temp2 = [x_dict['text'][i]]
        x_dict['text'][i] = str(temp1 + temp2)
    x['text'] = pd.DataFrame([x_dict['text']]).T 
    return x

train_2 = plus_catg(train_2,'digit_1')
test_2 = plus_catg(test_2, 'digit_1')

print(train_2.head())
print(test_2.head())

        AI_id  digit_1  digit_2  digit_3  \
0  id_0000001       19       95      952   
1  id_0000002        7       47      472   
2  id_0000003        7       46      467   
3  id_0000004        7       47      475   
4  id_0000005       17       87      872   

                                                text  
0              ['S', '▁ 카센터 / / 자동차부분정비 / 타이어 오일교환']  
1         ['G', '▁ 상점 내 / / 일반인 / / 대상 / / 채소과일 판매']  
2  ['G', '▁ 절단 / 사업체 / 도매 / 공업용 / 무 / 가지 / / 합성 /...  
3               ['G', '▁ 영업점 / / 일반소비자 / / 열쇠 잠금장치']  
4      ['Q', '▁ 어린이집 / 보호자 / / 위탁 / / 받아 / 취학전아동보육']  
       AI_id  digit_1 digit_2 digit_3  \
0  id_000001        9                   
1  id_000002        7                   
2  id_000003       19                   
3  id_000004       19                   
4  id_000005        9                   

                                             text  
0       ['I', '▁ 치킨전문점 / / 고객 / 주문 / / 해 / 치킨판매']  
1        ['G', '▁ 산업공구 / 다른 / 소매업자 / / 철물 / 수공구']  
2 

In [ ]:
data2= train_2['text']
target2 = train_2['digit_2']

# train_test_split
x_train2, x_val2, y_train2, y_val2 = train_test_split(data2, target2, test_size = 0.1, random_state=12)
test_train2 = test_2['text']

print(x_train2.head(3))
print(test_train2.head(3))

34488     ['G', '▁ 영업장 / / 일반대중 / / 대상 / / 각종 / 생필품 / 소매']
395706             ['P', '▁ 영어학원 / 초 / 중 / 고 / 대상 / 영어교육']
120781                     ['C', '▁ 반찬류 / 제조 / 판매 / 각종나물']
Name: text, dtype: object
0    ['I', '▁ 치킨전문점 / / 고객 / 주문 / / 해 / 치킨판매']
1     ['G', '▁ 산업공구 / 다른 / 소매업자 / / 철물 / 수공구']
2        ['S', '▁ 절 / / 신도을 / 대상 / / 불교단체 운영']
Name: text, dtype: object


In [ ]:
# Tokenizing

vocab2 = Counter()
x_dict2 = x_train2.to_dict()

for c in tqdm(x_dict2.keys()):
  x_dict2[c] = word_tokenize(x_dict2[c])
  for word in x_dict2[c]:
    vocab2[word] = vocab2[word] +1
vocab2_sorted = sorted(vocab2.items(), key=lambda x:x[1], reverse=True)
print(vocab2_sorted)

word_to_index = {}
i = 0
for (word, frequency) in vocab2_sorted:
  if frequency > 3 and frequency< 700000:
    i += 1
    word_to_index[word] = i
print(word_to_index)

x_train2 = tokenizing(x_train2)
x_val2 = tokenizing(x_val2)
test_train2 = tokenizing(test_train2)

print(x_train2)
print(x_val2)
print(test_train2)

100%|██████████| 900000/900000 [02:08<00:00, 7019.76it/s]


[('/', 4679282), ("'", 1800000), (',', 900827), ('[', 900000), (']', 900000), ("'▁", 899238), ("'G", 221882), ("'I", 168642), ('대상', 158155), ('갖추고', 125333), ('사업장', 116146), ('접객시설', 104304), ('고객', 102918), ("'C", 94736), ("'S", 90396), ("'H", 88224), ('일반인', 87354), ('매장', 72847), ('음식점', 61315), ('일반소비자', 59540), ('요청', 58283), ('판매', 57369), ('소매', 52213), ("'P", 41885), ('화물운송서비스', 40846), ('도매', 36187), ("'L", 36127), ('개인택시', 34135), ('승객운송서비스', 34118), ("'Q", 32478), ("'F", 31503), ('서비스', 29333), ('제공', 27213), ("'R", 26799), ('영업장', 26295), ('일반인대상', 26286), ("'M", 25632), ('용달화물자동차', 23502), ('사무실', 23322), ('등', 22420), ('제조', 20494), ('학원', 19244), ('받아', 18440), ('따라', 16542), ('계약', 16088), ("'N", 15891), ('가게', 15408), ('개별화물자동차', 15337), ('소매업자', 14917), ('부동산', 14226), ('미용실', 14001), ('일반고객', 13771), ('커피', 13404), ('해', 12861), ('교회', 11873), ('소비자', 11853), ('일반', 11645), ('시설', 11588), ('주점', 11300), ('가공', 10272), ('산업사용자', 10221), ('종교서비스', 9939), ("'J", 9812)

100%|██████████| 900000/900000 [02:16<00:00, 6593.92it/s]


                                        0
34488      [1, 29, 177, 3, 128, 1411, 17]
395706  [18, 1239, 390, 414, 434, 3, 253]
120781           [8, 2287, 35, 16, 11095]
684091             [2, 29, 7, 3, 108, 27]
157096             [25, 121, 89, 48, 923]
...                                   ...
564465                      [1, 2150, 20]
412925                  [2, 567, 895, 16]
255366              [18, 36, 60, 3, 2524]
759451              [8, 33, 91, 3773, 91]
472907   [1, 67, 14, 294, 536, 1117, 921]

[900000 rows x 1 columns]
853800            ['I', '▁ 사업장 / / 일반인대상 / / 김치전골 / 청국장판매']
542136           ['M', '▁ 조세 / 세무관련업무 / 의뢰 / / 받아서 / 기장대리']
239719                       ['I', '▁ 사업장 / / 시설갖추고 / 해산물']
557263       ['K', '▁ 보험회사 / / 일반인 / / 대상 / / 자동차보험 판매 중개']
891459                ['G', '▁ 화원 / 일반인 / / 대상 / / 꽃 / 화분']
                                ...                        
112252                    ['I', '▁ 주점 / / 접객요원 두 / / 맥주양주']
236781    ['C', '▁ 폴리 / 틸렌 PE 를가지 / / 절단성형 / / 해상

100%|██████████| 100000/100000 [00:15<00:00, 6641.19it/s]


                                                        0
853800                           [2, 5, 30, 13872, 13559]
542136                     [31, 1711, 1293, 85, 359, 590]
239719                                 [2, 5, 1492, 1210]
557263                     [63, 935, 11, 3, 5504, 16, 64]
891459                          [1, 714, 11, 3, 262, 870]
...                                                   ...
112252                             [2, 53, 86, 966, 1631]
236781  [8, 961, 8407, 1349, 7533, 11207, 3007, 589, 6...
212026                              [1, 195, 16, 29, 325]
200198  [25, 2120, 683, 1418, 140, 65, 37, 135, 9610, ...
425136                [1, 118, 942, 2114, 1000, 2017, 16]

[100000 rows x 1 columns]
0             ['I', '▁ 치킨전문점 / / 고객 / 주문 / / 해 / 치킨판매']
1              ['G', '▁ 산업공구 / 다른 / 소매업자 / / 철물 / 수공구']
2                 ['S', '▁ 절 / / 신도을 / 대상 / / 불교단체 운영']
3                     ['S', '▁ 영업장 / / 고객요구 / / 자동차튜닝']
4        ['I', '▁ 실내포장마차 / / 접객시설 / / 갖추고 / 소주 / 맥주제공

100%|██████████| 100000/100000 [00:15<00:00, 6636.79it/s]


                                    0
0           [2, 369, 7, 132, 48, 368]
1      [1, 17100, 404, 43, 276, 1595]
2        [9, 332, 21080, 3, 6190, 80]
3                 [9, 29, 156, 12337]
4           [2, 6338, 6, 4, 88, 1618]
...                               ...
99995          [1, 5, 30, 1673, 1676]
99996              [24, 172, 96, 349]
99997         [1, 51, 333, 50, 13499]
99998          [28, 5, 11, 151, 6785]
99999     [21, 5, 142, 3246, 141, 94]

[100000 rows x 1 columns]
                                        0
34488      [1, 29, 177, 3, 128, 1411, 17]
395706  [18, 1239, 390, 414, 434, 3, 253]
120781           [8, 2287, 35, 16, 11095]
684091             [2, 29, 7, 3, 108, 27]
157096             [25, 121, 89, 48, 923]
...                                   ...
564465                      [1, 2150, 20]
412925                  [2, 567, 895, 16]
255366              [18, 36, 60, 3, 2524]
759451              [8, 33, 91, 3773, 91]
472907   [1, 67, 14, 294, 536, 1117, 921]

[900000 rows

In [ ]:
max_features = 24000
text_max_words = 20

# Preprocessing
x_train2 = sequence.pad_sequences(x_train2[0], maxlen=text_max_words)
x_val2 = sequence.pad_sequences(x_val2[0], maxlen=text_max_words)
y_train2 = np_utils.to_categorical(y_train2)
y_val2 = np_utils.to_categorical(y_val2)

print(x_train2)
print(x_val2)
print(y_train2)
print(y_val2)

[[    0     0     0 ...   128  1411    17]
 [    0     0     0 ...   434     3   253]
 [    0     0     0 ...    35    16 11095]
 ...
 [    0     0     0 ...    60     3  2524]
 [    0     0     0 ...    91  3773    91]
 [    0     0     0 ...   536  1117   921]]
[[    0     0     0 ...    30 13872 13559]
 [    0     0     0 ...    85   359   590]
 [    0     0     0 ...     5  1492  1210]
 ...
 [    0     0     0 ...    16    29   325]
 [    0     0     0 ...   135  9610   140]
 [    0     0     0 ...  1000  2017    16]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# Predict
pred = bilstm_model(97, 2, x_train2, y_train2, x_val2, y_val2)
pred_2 = pred.tolist()
pred_2 = pd.DataFrame(data= pred_2, columns=['digit_2'])

pred_2.to_csv('/content/drive/MyDrive/2022/공모전/digit2_1.csv')

In [ ]:
# Todo: Tokenizing & Learning - digit_3
'''digit_2을 예측한 뒤 digit_3 예측에 활용함'''

train_3 = train.copy()
test_3 = test.copy()
test_3['digit_2'] = pred_2['digit_2']
print(train_3.head())
print(test_3.head())

In [ ]:
def plus_catg2(x,num):
        # digit_?을 text에 append하는 함수
    x_dict = x.to_dict()
    for i in range(0,len(x_dict['text'])):
        temp1 = [x_dict[num][i]]
        temp2 = [x_dict['text'][i]]
        x_dict['text'][i] = str(temp1 + temp2)
    x['text'] = pd.DataFrame([x_dict['text']]).T 
    return x

train_3 = plus_catg2(train_3,'digit_2')
test_3 = plus_catg2(test_3, 'digit_2')

print(train_3.head())
print(test_3.head())

In [ ]:
data= train_3['text']
target = train_3['digit_3']

# train_test_split
x_train3, x_val3, y_train3, y_val3 = train_test_split(data, target, test_size = 0.1, random_state=12)
test_train3 = test_3['text']

In [ ]:
# Tokenizing
vocab = Counter()
x_dict3 = x_train3.to_dict()
vocab_sorted = {}
for c in tqdm(x_dict3):
    x_dict3[c] = word_tokenize(x_dict3[c])
    for word in x_dict3[c]:
        vocab[word] = vocab[word] +1
vocab_sorted = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
print(vocab_sorted)

word_to_index = {}
i = 0
for (word, frequency) in vocab_sorted:
    if frequency > 3 and frequency< 700000:
        i += 1
        word_to_index[word] = i
print(word_to_index)

In [ ]:
def tokenizing(x):
    print(x)
    x_dict = x.to_dict()
    for c in tqdm(x_dict):
        x_dict[c] = word_tokenize(x_dict[c])
        temp = []
        for w in x_dict[c]:
            try:
                temp.append(word_to_index[w])
            except KeyError:
                pass
        x_dict[c] = temp
    x = pd.DataFrame([x_dict]).T
    print(x)
    return x
x_train3 = tokenizing(x_train3)
x_val3 = tokenizing(x_val3)
test_train3 = tokenizing(test_train3)

In [ ]:
max_features = 24000
text_max_words = 20

# Preprocessing
x_train3 = sequence.pad_sequences(x_train3[0], maxlen=text_max_words)
x_val3 = sequence.pad_sequences(x_val3[0], maxlen=text_max_words)
y_train3 = np_utils.to_categorical(y_train3)
y_val3 = np_utils.to_categorical(y_val3)

# Predict
pred = bilstm_model(970, 3, x_train3, y_train3, x_val3, y_val3)
pred_3 = pred.tolist()
pred_3 = pd.DataFrame(data= pred_3, columns=['digit_3'])

pred_3.to_csv('/content/drive/MyDrive/2022/공모전/digit3_1.csv')
print(pred_3.head())

In [ ]:
# Todo: Predict File
test = pd.read_csv('/content/drive/MyDrive/test.csv', encoding='cp949')
test['digit_2'] = pred_2['digit_2']
test['digit_1'] = pred_1['digit_1']
test['digit_3'] = pred_3['digit_3']

test.to_csv('/content/drive/MyDrive/test_submit1.csv')